# Compare analytical and autodiffed stiffness matrix

Here we compare how much autodiffed solution is slower than analytical.

Author(s): Jukka Aho <jukka.aho@kapsi.fi>

Last updated:

In [1]:
using Dates
today()

2015-06-15

In [2]:
using ForwardDiff
ENV["COLUMNS"] = 160

160

In [3]:
# Partial derivatives of bilinear Lagrange polynomials
dNdξ(ξ) = [[-(1-ξ[2])/4.0    -(1-ξ[1])/4.0],
           [ (1-ξ[2])/4.0    -(1+ξ[1])/4.0],
           [ (1+ξ[2])/4.0     (1+ξ[1])/4.0],
           [-(1+ξ[2])/4.0     (1-ξ[1])/4.0]]    

a = 1/sqrt(3)
ipoints = [[-a -a], [a -a], [a a], [-a a]]
iweights = [1 1 1 1]

E = 90
ν = 0.25
μ = E/(2*(1+ν))
λ = E*ν/((1+ν)*(1-2*ν))
λ = 2*λ*μ/(λ + 2*μ)
μ, λ

(36.0,24.0)

## Version using automatic differentiation

In [4]:
function calc_local_matrices!(X, u, R, Kt; dim=2)
    I = eye(dim)
    
    function calc_Wint!(u, Wint)
        for m = 1:length(iweights)
            w = iweights[m]
            ξ = ipoints[m, :]
            Jᵀ = X*dNdξ(ξ)
            ∇N = inv(Jᵀ)*dNdξ(ξ)'
            ∇u = u*∇N'
            F = I + ∇u  # Deformation gradient
            E = 1/2*(∇u' + ∇u + ∇u'*∇u)  # Green-Lagrange strain tensor
            S = λ*trace(E)*I + 2*μ*E  # PK2 stress tensor
            P = F*S  # PK1 stress tensor
            Wint[:,:] += w*P*∇N*det(Jᵀ)
        end
    end

    # herlper for tangent stiffness matrix
    function R!(u, R)
        R[:] = 0
        calc_Wint!(reshape(u, 2, 4), reshape(R, 2, 4))
        #calc_Wext!(reshape(u, 2, 4), reshape(R, 2, 4))
    end
    Jacobian = ForwardDiff.forwarddiff_jacobian(R!, Float64, fadtype=:dual, n=8, m=8)

    Kt[:,:] = Jacobian(reshape(u, 8))
    R!(reshape(u, 8), reshape(R, 8))
end

calc_local_matrices! (generic function with 1 method)

In [5]:
# validation
X = [0 0; 10 0; 10 1; 0 1]'
u = zeros(2,4)
R = zeros(2,4)
Kt = zeros(8,8)

free_dofs = [3, 4, 5, 6]
for i in 1:10
    calc_local_matrices!(X, u, R, Kt)
    R[2,3] += 2
    du = Kt[free_dofs, free_dofs] \ -reshape(R, 8)[free_dofs]
    u[free_dofs] += du
    if norm(du) < 1.0e-9
        println("Converged in ", i, " iterations")
        break
    end
end
println(u)

Converged in 6 iterations
[0.0 -0.3991450609547433 -0.07228582695592461 0.0
 0.0 -2.1779892317073504 -2.222244754401764 0.0]


In [6]:
function test_algo1(N=10000)
    for i=1:N
        calc_local_matrices!(X, u, R, Kt)
    end
end
test_algo1()

In [7]:
@time test_algo1()

elapsed time: 11.336512664 seconds (2020893880 bytes allocated, 26.15% gc time)


## Analytical tangent stiffness

In [8]:
function calc_local_matrices2!(X, u, R, Kt; dim=2)
    I = eye(dim)
    R[:,:] = 0.0
    Kt[:,:] = 0.0
    N = 4 # number of shape functions

    dF = zeros(2, 2)

    for m = 1:length(iweights)
        w = iweights[m]
        ξ = ipoints[m, :]
        Jᵀ = X*dNdξ(ξ)
        detJ = det(Jᵀ)
        ∇N = inv(Jᵀ)*dNdξ(ξ)'
        ∇u = u*∇N'
        F = I + ∇u  # Deformation gradient
        E = 1/2*(∇u' + ∇u + ∇u'*∇u)  # Green-Lagrange strain tensor
        S = λ*trace(E)*I + 2*μ*E  # PK2 stress tensor
        P = F*S  # PK1 stress tensor
        R[:,:] += w*P*∇N*detJ

        for p = 1:N
            for i = 1:dim
                dF[:,:] = 0.0
                dF[i,:] = ∇N[:,p]
                dE = 1/2*(F'*dF + dF'*F)
                dS = λ*trace(dE)*I + 2*μ*dE
                dP = dF*S + F*dS
                for q = 1:N
                    for j = 1:dim
                        Kt[dim*(p-1)+i,dim*(q-1)+j] += w*(dP[j,:]*∇N[:,q])[1]*detJ
                    end
                end
            end
        end

    end
end

calc_local_matrices2! (generic function with 1 method)

In [9]:
# validation
X = [0 0; 10 0; 10 1; 0 1]'
u = zeros(2,4)
R = zeros(2,4)
Kt = zeros(8,8)

free_dofs = [3, 4, 5, 6]
for i in 1:10
    calc_local_matrices2!(X, u, R, Kt)
    R[2,3] += 2
    du = Kt[free_dofs, free_dofs] \ -reshape(R, 8)[free_dofs]
    u[free_dofs] += du
    if norm(du) < 1.0e-9
        println("Converged in ", i, " iterations")
        break
    end
end
println(u)

Converged in 6 iterations
[0.0 -0.39914506095474317 -0.07228582695592449 0.0
 0.0 -2.1779892317073504 -2.222244754401764 0.0]


In [10]:
function test_algo2(N=10000)
    for i=1:N
        calc_local_matrices2!(X, u, R, Kt)
    end
end
test_algo2()

In [11]:
@time test_algo2()

elapsed time: 8.447644445 seconds (1554800080 bytes allocated, 29.45% gc time)
